In [12]:
#Grab in the specific helper libraries we would like

#notes, non-filtering by ID, push to csv
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import csv



INPUT_DIR = os.getcwd() + "\\..\\data\\input"
OUTPUT_DIR = os.getcwd() + "\\..\\data\\output"

TEST_FILE_NAME = INPUT_DIR + "\\Interactions 01_26_2022 after 10_31_2021.xlsx"


In [37]:
#make sure we 
print(TEST_FILE_NAME)

c:\dev\Gerontechnology\data wrangling\src\..\data\input\Interactions 01_26_2022 after 10_31_2021.xlsx


In [38]:
df = pd.read_excel(TEST_FILE_NAME)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

,interactionId,participantId,elementId,timestamp_local,timestamp_utcoffset,interaction,token,type
0,1,48,69,2021-10-15 17:07:19,-500,tap,NaN,NaN
1,2,48,3,2021-10-15 17:07:20,-500,present,NaN,NaN
2,3,48,13,2021-10-15 17:07:28,-500,tap,NaN,NaN
3,4,48,238,2021-10-15 17:07:30,-500,tap,NaN,NaN
4,5,48,90,2021-10-15 17:07:30,-500,present,NaN,NaN
...,...,...,...,...,...,...,...,...
186489,186490,110,333,2022-01-26 00:48:21,-800,tap,NaN,NaN
186490,186491,110,334,2022-01-26 00:48:51,-800,tap,NaN,NaN
186491,186492,110,340,2022-01-26 00:49:15,-800,tap,NaN,NaN
186492,186493,110,44,2022-01-26 00:49:20,-800,tap,NaN,NaN


In [5]:

d_str = '1/4/2022 9:54'
def conv_t_hard(d_str):
    #print(d_str)
    #date,other = d_str.split('  ')
    date,curr_time = d_str.split(' ')
    #curr_time, ampm = other.split(' ')
    month,day,year = date.split('/')
    month = int(month)
    year = int(year)
    day = int(day)
    hour,minute = curr_time.split(':')
    hour = int(hour)
    minute = int(minute)
    j = 0
    month_secs = 0
    while j <= month:
        if(j == 2):
            month_secs += 2419200
        elif(j == 4 or j == 6 or j == 9 or j == 11):
            month_secs += 2592000
        else:
            month_secs += 2678400
        j += 1
    secs = (year-1970)*31536000 + month_secs + day*24*60*60 + hour*60*60 + minute*60
    return(secs)
out = conv_t_hard(d_str)
print(out)

1645610040


In [39]:
def conv_time(t_string, day):
    #print(t_string)
    time,dmark = t_string.split(' ')
    hour,minutes = time.split(':')
    offset = 0
    if(dmark == 'PM'):
        offset += 12*60*60
    secs = 60*60*int(hour) + 60*int(minutes) + offset +int(day)*24*60*60
    return(secs)
sec = conv_time('5:41 PM', 2)
print(sec)
    

236460


In [40]:
def cal_by_day(day, sec_gap, parc_time, parc_occur,df):
    '''
    we want two dicts: one that holds the last occurance of each participant,
    and one that holds the number of times a gap of size n secs has occured
    '''
    ids = df['participantId_'+str(day)].dropna()
    times = df['Time_'+str(day)].dropna()
    i = 0
    for rows in ids:
        curr_id = ids[i]
        curr_time = times[i]

        if curr_id in parc_time.keys():
            old_time = parc_time.get(curr_id)
            new_time = times[i]
            new_secs = conv_time(new_time, day)
            if(new_secs - old_time >= sec_gap):
                if curr_id in parc_occur:
                    curr_count = parc_occur.get(curr_id)
                    parc_occur.update({curr_id:curr_count+1})
                else:
                    parc_occur.update({curr_id:1})
            parc_time.update({curr_id:new_secs})
        else:
            new_time = times[i]
            new_secs = conv_time(new_time, day)
            parc_time.update({curr_id:new_secs})

        i += 1
    return(parc_time, parc_occur)

In [41]:
print(df["Time_3"].dropna())

KeyError: 'Time_3'

In [ ]:
day = 20
sec_gap = 200
end_day = 23
parc_ids = [95,106] #the ids we actually care about, leave empty if we want all
event_ids = []

parc_time = {}
parc_occur = {}

while day <= end_day:
    parc_time, parc_occur = cal_by_day(day, sec_gap, parc_time, parc_occur, df)
    if(len(parc_ids) == 0):
        print("----------------------------")
        print("----------------------------")
        print("For Day:" + str(day))
        print("Times are:")
        print(parc_time)

        print("Occurrences are:")
        print(parc_occur)

        print("----------------------------")
        print("----------------------------")
        
    day += 1
    


In [ ]:
for p in parc_ids:
    print("Number of unique occurences:")
    occ = parc_occur.get(p)
    print(p)

Number of unique occurences:
95
Number of unique occurences:
106


In [ ]:
df_long = pd.read_csv('./Documents/GitHub/hydraGAN_code_time_series/all_long.csv')

In [ ]:
def break_by_day(d_str,curr_day):
    month,day,dontcare = d_str.split('/')
    if(int(day)!= curr_day):
        return(1)
    return(0)
def get_current_day(d_str):
    #print(d_str)
    month,day,dontcare = d_str.split('/')
    return(int(day))

def cal_by_day_single(day, sec_gap, parc_time, parc_occur,df,el_ids,curr_day):
    '''
    we want two dicts: one that holds the last occurance of each participant,
    and one that holds the number of times a gap of size n secs has occured
    '''
    ids = df['participantId'].dropna()
    times = df['timestamp_local'].dropna()
    i = 0
    for rows in ids:
        curr_id = ids[i]
        curr_time = times[i]

        if curr_id in parc_time.keys():
            old_time = parc_time.get(curr_id)
            new_time = times[i]
            new_secs = conv_t_hard(new_time)
            if(new_secs - old_time >= sec_gap):
                if curr_id in parc_occur:
                    if(len(el_ids) == 0 or df['elementId'][i] in el_ids):
                        curr_count = parc_occur.get(curr_id)
                        parc_occur.update({curr_id:curr_count+1})
                else:
                    parc_occur.update({curr_id:1})
            parc_time.update({curr_id:new_secs})
        else:
            new_time = times[i]
            new_secs = conv_t_hard(new_time)
            parc_time.update({curr_id:new_secs})

        i += 1
    return(parc_time, parc_occur)

In [ ]:
def filter_df_to_correct_days(df, first_day, end_day):
    #print(df.loc[0, 'timestamp_local'])
    
    new_df = pd.DataFrame()
    #new_df.columns = df.columns
    f_day = df.loc[0, 'timestamp_local']
    current_date = get_current_day(f_day)
    curr_d_index = 1
    i = 0
    while i < df.shape[0]:
        new_date = get_current_day(df.loc[i, 'timestamp_local'])
        if(new_date != current_date):
            curr_d_index += 1
            current_date = new_date 
        #print(curr_d_index, first_day)
        if(int(curr_d_index) >= first_day and int(curr_d_index) <= end_day):
            #new_df.append(df.loc[i])
            new_df = pd.concat([new_df, df.loc[i]],axis=1)
        i += 1
    return(new_df.T)
    
new_df = filter_df_to_correct_days(df_long, 3,4)
#print(df_long.shape)
print(new_df.head(3))

    interactionId participantId elementId  timestamp_local  \
225           226            48       263  10/18/2021 9:13   
226           227            48         8  10/18/2021 9:13   
227           228            48         3  10/18/2021 9:13   

    timestamp_utcoffset interaction token type  
225                -700         tap   NaN  NaN  
226                -700         tap   NaN  NaN  
227                -700     present   NaN  NaN  


In [ ]:
#put access calculations here:
    

In [ ]:
df_long = pd.read_csv('./Documents/GitHub/hydraGAN_code_time_series/all_long.csv') #Put the path to the csv you care about here
start_day = 1 # day to start
sec_gap = 20 #how many seconds between interactions before we care about it
end_day = 2 #day to end
parc_ids = [] #the ids we actually care about, leave empty if we want all
elem_ids = [] #the elements we want, leave blank if we want all

parc_time = {}
parc_occur = {}

'''
filter by day starting day

dict to csv (locally)
'''
#curr_day = get_current_day(df_long['timestamp_local'][0])

#print("We are starting on day:", curr_day)
df_long = filter_df_to_correct_days(df_long,start_day,end_day)


parc_time, parc_occur = cal_by_day_single(day, sec_gap, parc_time, parc_occur, df_long,elem_ids, curr_day)
if(len(parc_ids) == 0):
    print("----------------------------")
    print("----------------------------")
    #print("For Day:" + str(day))
    print("Times are:")
    #print(parc_time)

    print("Occurrences are:")
    print(parc_occur)

    print("----------------------------")
    print("----------------------------")
       

----------------------------
----------------------------
Times are:
Occurrences are:
{48: 14, 75: 1}
----------------------------
----------------------------


In [ ]:
with open('./output.csv', 'w') as csvfile:
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerow(['Participant', 'Count'])
    for key,value in parc_occur.items():
        print(key,value)
        csvwriter.writerow([key, value])

48 14
75 1


In [29]:
def next_sunday(original_datetime):
    """Returns the date of the next Sunday relative to the given date.

    Args:
        original_datetime (Datetime): A date

    Returns:
        Datetime: This will be a Sunday datetime.
    """
    days_left = 7 - (original_datetime.floor(freq='D').day_of_week + 1) # offset needed because day of week starts on monday
    if(days_left == 0):
        days_left = 7
    return (pd.to_datetime(original_datetime) + timedelta(days=days_left)).floor(freq='D')

In [30]:
def get_interaction_counts(df, elementIDs, day_of_week=-1):
    """Given a DataFrame which contains participant interactions,
    returns a count of interactions for the given elementID for each participant.

    Args:
        df (Pandas DataFrame): Created earlier from read_excel
        elementIDs (list): A list of elementIDs where each elementID number is to be aggregated.
        day_of_week (int, optional): Discriminates further on query to only when its this day of the week (Sunday = 0, Saturday = 6)
    """
    query_string = ""

    if(day_of_week >= 0):
        # Filter df to only include interactions on this day of the week
        end_date = next_sunday(df['timestamp_local'].max()) - timedelta(days=(6-day_of_week))
        start_date = end_date - timedelta(days=1)
        df = df.loc[(df["timestamp_local"] >= start_date) & (df["timestamp_local"] < end_date)]


    for i in range(len(elementIDs)):
        query_string += 'elementId == {}'.format(elementIDs[i])
        if((i+1) < len(elementIDs)):
            query_string += " or "
    
    count = df.query(query_string)
    grouping1 = count.groupby(['participantId', 'elementId']).size()
    
    return grouping1.groupby(['participantId']).sum()
    


In [31]:
def create_variable(participants_dict, interactions_df, variable_name, elementIDs, variable_func, day_of_week=-1):
    """Creates a new variable and calculates it, storing the results for each participant in a dictionary.

    Args:
        participants_dict (Dict): Participant data, key = participantID, value = dict of variables
        interactions_df (DataFrame): An already created DataFrame that is data collected from some excel file
        variable_name (string): Identifier by which this variable is known as.
        elementIDs (list): A list of elementIDs that are counted and summed for this variable.
        variable_func (Lambda Function): Describes how the aggregated count will be calculated, typically division due to weekly or daily averages.

    Returns:
        Dict: An updated dictionary containing 
    """
    # Each interaction is a pair ([participantID, count)
    element_count_list = get_interaction_counts(interactions_df, elementIDs, day_of_week=day_of_week).iteritems()

    for element_count in element_count_list:
        participant_id = element_count[0]

        if participant_id in participants_dict:
            participants_dict[participant_id][variable_name] = variable_func(element_count[1])
        else:
            participants_dict[participant_id] = dict()
            participants_dict[participant_id][variable_name] = variable_func(element_count[1])
    
    for participant_id in participants_dict:
        if variable_name not in participants_dict[participant_id]:
            participants_dict[participant_id][variable_name] = 0
    
    return participants_dict



In [32]:
def participant_dict_to_csv(participants, outfile_name="output.csv"):
    """Given an instantiated dictionary, converts to a DataFrame to then be translated to a csv file.

    Args:
        participants (Dict): Participant data, key = participantID, value = dict of variables
        outfile_name (str, optional): The name of the file to be write to. Relative path, using global OUTPUT_DIR. Defaults to "output.csv".
    """
    interactions_df = pd.DataFrame.from_dict(participants, orient='index')
    interactions_df.reset_index(inplace=True)
    interactions_df.rename({'index':'participantId'}, axis='columns', inplace=True)
    interactions_df.to_csv(OUTPUT_DIR + "\\{}".format(outfile_name), index=False)

In [33]:
def get_weekly_interaction_dfs(interactions_filename):
    """Given a csv file, converts a dataset of interactions spanning many weeks into many datasets by week.

    Args:
        interactions_filename (string): Absolute filepath of excel file.

    Returns:
        Dict: Dictionary of interaction DataFrames. Key = StartDate, Value=DataFrame
    """
    weekly_dfs = dict()
    interactions = pd.read_excel(interactions_filename)

    start_date = interactions['timestamp_local'].min()
    end_parsec = next_sunday(start_date)
    end_date = interactions['timestamp_local'].max()

    total = 0
    while(start_date < end_date):
        df = interactions.loc[(interactions["timestamp_local"] >= start_date) & (interactions["timestamp_local"] < end_parsec)]
        weekly_dfs[(start_date, end_parsec)] = df        
        start_date = end_parsec
        end_parsec = next_sunday(start_date)
    return weekly_dfs

In [34]:
def get_variable_calculations(df, outfile_name):
    """Given a dataset of interactions from participants, computes and stores calculation dependent variables to an output file.

    Args:
        interactions_filename (string): Excel file where all interaction data is stored (absolute path)
        data_date_range (int, optional): The range of days by which this data was collected. Defaults to 7 (week's worth of data).
    """
    participants = dict()

    participants = create_variable(participants, df, "CalenderUse", [9], lambda x: x/(7))
    participants = create_variable(participants, df, "SumTotalCalendarInteractions", [9, 18, 19, 20], lambda x: x/(7))
    participants = create_variable(participants, df, "CalendaringGoal", [9], lambda x: x/(4))

    participants = create_variable(participants, df, "TodayPageUse-Sunday", [8, 13, 379, 380, 381, 384], lambda x: x, day_of_week=0)
    participants = create_variable(participants, df, "TodayPageUse-Monday", [8, 13, 379, 380, 381, 384], lambda x: x, day_of_week=1)
    participants = create_variable(participants, df, "TodayPageUse-Tuesday", [8, 13, 379, 380, 381, 384], lambda x: x, day_of_week=2)
    participants = create_variable(participants, df, "TodayPageUse-Wednesday", [8, 13, 379, 380, 381, 384], lambda x: x, day_of_week=3)
    participants = create_variable(participants, df, "TodayPageUse-Thursday", [8, 13, 379, 380, 381, 384], lambda x: x, day_of_week=4)
    participants = create_variable(participants, df, "TodayPageUse-Friday", [8, 13, 379, 380, 381, 384], lambda x: x, day_of_week=5)
    participants = create_variable(participants, df, "TodayPageUse-Saturday", [8, 13, 379, 380, 381, 384], lambda x: x, day_of_week=6)


    participants = create_variable(participants, df, "SumTotalEventInteractions", [13, 103, 104, 379, 380, 381, 384], lambda x: x/(7))

    participant_dict_to_csv(participants, outfile_name=outfile_name)


In [35]:
def create_weekly_calculations_table(interactions_filename):
    """Given a large dataset of interactions from multiple participants, outputs all weekly csv files from that data showcasing desired calculated variables.

    Args:
        interactions_filename (string): Absolute path to large dataset
    """
    weekly_dfs = get_weekly_interaction_dfs(interactions_filename)

    for date_range, df in weekly_dfs.items():
        date = date_range[0].isocalendar()
        filename = "Week {}, {}".format(date[1], date[0])
        get_variable_calculations(df, "{}.csv".format(filename))
    
  

In [36]:
# given test file, all interaction variable calculations will go to output directory.
create_weekly_calculations_table(TEST_FILE_NAME)